<a href="https://colab.research.google.com/github/pradhyumna2003/TourismChatbot-LLM/blob/main/TouristChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers

### Get HUGGINGFACEHUB_API_KEY

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "API_KEY"

### Download Text File

In [ ]:
import requests

url = "https://raw.githubusercontent.com/pradhyumna2003/Thanjavur-TXT/main/Brihadeeswara%20Temple%20-%20Architecture.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [ ]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [ ]:
documents

[Document(page_content='3/8/24, 3:16 PM\n\nBrihadeeswara Temple - Architecture\n\nBrihadeeswara Temple - Architecture\nBrihadeeswara temple was constructed by Rajaraja Chola I and was designed by\nKunjara Mallan Raja Rama Perunthachan. The base of the temple has the\npictures of dancing Shiva and stone deities. A 2m high, 6m long, and 2.5m wide\nstatue of Nandi Bull weighing 20 tonnes can be found at the entrance.\nThe bull is made up of a single stone and is the second largest statue of Nandi. The\nShiva Linga inside the temple has the height of 3.7m. The dynasties that ruled after\nCholas also added the pictures of many deities.\n\nEntrance\nThe platform on which the temple was built has the height of 5m. The outer walls of\nthe temple are built like fort having a deep moat surrounding it. There are two\nentrances to the temple.\n\nThe first is the five storey gopuram and the second is the free standing gopuram to\nwhich people reach through a quadrangle. Vimanam or the Shikhara of t

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
print(wrap_text_preserve_newlines(str(documents[0])))

page_content='3/8/24, 3:16 PM\n\nBrihadeeswara Temple - Architecture\n\nBrihadeeswara Temple -
Architecture\nBrihadeeswara temple was constructed by Rajaraja Chola I and was designed by\nKunjara Mallan
Raja Rama Perunthachan. The base of the temple has the\npictures of dancing Shiva and stone deities. A 2m
high, 6m long, and 2.5m wide\nstatue of Nandi Bull weighing 20 tonnes can be found at the entrance.\nThe bull
is made up of a single stone and is the second largest statue of Nandi. The\nShiva Linga inside the temple has
the height of 3.7m. The dynasties that ruled after\nCholas also added the pictures of many
deities.\n\nEntrance\nThe platform on which the temple was built has the height of 5m. The outer walls of\nthe
temple are built like fort having a deep moat surrounding it. There are two\nentrances to the temple.\n\nThe
first is the five storey gopuram and the second is the free standing gopuram to\nwhich people reach through a
quadrangle. Vimanam or the Shikhara of the temple\

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

3

In [ ]:
docs[0]

Document(page_content='3/8/24, 3:16 PM\n\nBrihadeeswara Temple - Architecture\n\nBrihadeeswara Temple - Architecture\nBrihadeeswara temple was constructed by Rajaraja Chola I and was designed by\nKunjara Mallan Raja Rama Perunthachan. The base of the temple has the\npictures of dancing Shiva and stone deities. A 2m high, 6m long, and 2.5m wide\nstatue of Nandi Bull weighing 20 tonnes can be found at the entrance.\nThe bull is made up of a single stone and is the second largest statue of Nandi. The\nShiva Linga inside the temple has the height of 3.7m. The dynasties that ruled after\nCholas also added the pictures of many deities.\n\nEntrance\nThe platform on which the temple was built has the height of 5m. The outer walls of\nthe temple are built like fort having a deep moat surrounding it. There are two\nentrances to the temple.\n\nThe first is the five storey gopuram and the second is the free standing gopuram to\nwhich people reach through a quadrangle. Vimanam or the Shikhara of th

### Embeddings

In [ ]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [ ]:
!pip install faiss-cpu

In [ ]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)



In [ ]:
query = "brihadeeswara temple constructed by"
docs = db.similarity_search(query)

In [ ]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

Deities of the Temple
Lord Shiva is the main deity of the temple and a Shiva linga is established here to
worship him. The height of the Shiva linga is 8.7m. Other deities are placed in the
outer wall also known as Koshtha Moorthigal.

https://www.tutorialspoint.com/brihadeeswara_temple/brihadeeswara_temple_architecture.htm

2/3

 3/8/24, 3:16 PM

Brihadeeswara Temple - Architecture

These deities include Dakshinamurthy, Surya and Chandra. Other deities include
Ashta Dikpaalakas, Agni, Yam, Nirrti, Varuna, Vayu, Kuber, and Isana. The
statues of all these deities are six feet high.

https://www.tutorialspoint.com/brihadeeswara_temple/brihadeeswara_temple_architecture.htm

3/3


### Create QA Chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub


In [ ]:
llm=HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature":0, "max_length":512})

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "What did the president say about the Supreme Court"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

"It's a good court."

In [ ]:
query = "brihadeeswara temple constructed by?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Rajaraja Chola I'

### Working with PDF Files

In [ ]:
!pip install unstructured
!pip install chromadb
!pip install Cython
!pip install tiktoken
!pip install unstructured[local-inference]

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
pdf_folder_path = '/content/sample_data/pdf/'
os.listdir(pdf_folder_path)

['Brihadeeswara Temple - Nearby Places.pdf',
 'Temple Towns and Pilgrimage Centres (Thanjavur).pdf',
 'brihadeeswara_temple.pdf',
 'Brihadeeswara Temple - Architecture.pdf']

In [ ]:
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
loaders

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)).from_loaders(loaders)

In [ ]:
llm=HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature":0, "max_length":512})

In [ ]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=index.vectorstore.as_retriever(),
                                    input_key="question")

In [ ]:
chain.run('tell me about Gangaikonda Cholapuram?')

'Rajendra Chola I, son and successor of Rajaraja Chola I, constructed Gangaikonda Cholapuram.'

In [ ]:
chain.run('Any nearby hotel?')

'Five-Star Hotels o Hotel Svatma located at Blake Higher Secondary Road Four-Star Hotels o Lakshmi Hotel located at VOC Nagar, Trichy Main Road 4 Brhadeeswara Temple Three-Star Hotel o Hotel Balaji Inn located near Bus Stand o Hotel Parisutham located at GA Canal Road o Hotel Gnanam located at Anna Salai o Hotel Sangam located at Trichy o Hotel Victoriyah located near Rohini Hospital Budget Hotels or Two-Star Hotels o Hotel Abis Inn located at Pudukottai Main Road o Hotel Ramnath located near Bus Stand'

In [ ]:
chain.run('Visiting hours?')

'6:00am to 12:30pm and 4:00pm to 8:30pm'

In [ ]:
chain.run('tell me about thanjavur?')

'The city of Thanjavur is situated on the banks of river Krishna and it became the biggest pilgrimage site. People would come here to worship and they were intoxicated by the marvellous architecture of the city. Apart from the temple, the city had palaces with mandapas or platforms, from where the king held the courts and issued orders. There were barracks for the army stationed near the palace. Another major feature of this town was the market. It was filled with traders selling food grains like wheat and millet. Spices, Jewellery and Clothes were the major items in the market. Thanjavur played a crucial role in influencing the economy of nearby villages and towns. The Saliya weavers of Thanjavur and Uraiyur produced fine cotton clothes for the king and nobles and also for the public. Weavers also produce clothes to make flags for ceremonies in the temple.'

In [ ]:
chain.run('what happend when pandyas ruled thanjavur?')

'he captured Thanjavur two times'